## 명목임금지수
## https://bigdata-sme.kr/#/datastore/detail/statistics/DATASET_KPC_14

In [ ]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
import time # API 서버 부하를 줄이기 위해 time 라이브러리 추가

# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("wage_key")
if not api_key:
    print("❌ 오류: .env 파일에서 'wage_key' 키를 찾을 수 없습니다.")
else:
    url = "https://dataapi.wehago.com/data/"
    headers = {
        "apicode": api_key
    }
    
    # --- 반복문 로직 ---
    all_data_list = []  # 모든 데이터를 저장할 빈 리스트 (장바구니)
    start_index = 0
    chunk_size = 500    # 한 번에 요청할 개수 (0~499는 500개)

    print("데이터 수집을 시작합니다...")

    while True:
        # 1. 'from'과 'to' 파라미터를 동적으로 설정
        params = {
            "dataset_key": "DATASET_SUNDO_11",
            "from": start_index,
            "to": start_index + chunk_size - 1
        }
        
        print(f"  -> {start_index}부터 {start_index + chunk_size - 1}까지 요청 중...")

        try:
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                
                # 2. 'data' 키에서 실제 데이터 리스트를 가져옴
                data_chunk = data.get('data') # .get()을 사용하면 'data' 키가 없어도 오류 안남

                # 3. 데이터가 비어있는지 확인
                if not data_chunk:
                    # 서버가 빈 리스트([])를 반환하면, 더 이상 데이터가 없다는 의미
                    print("✅ 모든 데이터를 수집했습니다. 반복을 종료합니다.")
                    break
                
                # 4. (중요) 가져온 데이터를 '장바구니' 리스트에 추가
                all_data_list.extend(data_chunk)
                
                # 5. 다음 루프를 위해 시작 인덱스 업데이트
                start_index += chunk_size
                
                # (권장) API 서버에 대한 예의로, 요청 사이에 짧은 휴식 추가
                time.sleep(0.1) # 0.1초 대기

            else:
                # API 호출 자체가 실패한 경우 (400, 404, 500 등)
                print(f"❌ 오류 발생 (Status Code: {response.status_code})")
                print(response.text)
                break # 오류 발생 시 반복 중단

        except requests.exceptions.RequestException as e:
            print(f"연결 중 오류가 발생했습니다: {e}")
            break # 오류 발생 시 반복 중단

    # --- 반복문 종료 후 ---
    if all_data_list:
        # 6. '장바구니'에 모아둔 모든 데이터를 하나의 DataFrame으로 변환
        df_total = pd.DataFrame(all_data_list)
        
        print("\n--- 최종 DataFrame 변환 결과 ---")
        print(f"총 {len(df_total)}개의 행(row)을 가져왔습니다.")
        print(df_total.head())
    else:
        print("\n가져온 데이터가 없습니다.")

In [ ]:
df_total.to_csv('명목임금지수.csv')